In [ ]:
# P Naidoo

In [1]:
!pip install scipy
!pip install pandas
!pip install shutil
!pip install opencv-python
!pip install sklearn
!apt update && apt install -y libsm6 libxext6
!apt apt-get install -y libxrender-dev
!apt-get update
!apt-get install ffmpeg libsm6 libxext6  -y

import os
import shutil
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import clear_output

import random

import cv2

LOCAL_DATA_PATH = '/tf/notebooks/Data/echonet frames split for model training'


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease                
Hit:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease              
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1581 B]
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease  

In [2]:
DATA_PATH = LOCAL_DATA_PATH
train_dir = os.path.join(LOCAL_DATA_PATH, 'images/training')
test_dir = os.path.join(LOCAL_DATA_PATH, 'images/testing')
val_dir = os.path.join(LOCAL_DATA_PATH, 'images/validation')
annot_dir = os.path.join(LOCAL_DATA_PATH, 'annotations_binary')


# Settings

In [3]:

OUTPUT_CHANNELS = 1 # no of classes
BATCH_SIZE = 16      # 8 for GPU Server
BUFFER_SIZE = 1000
EPOCHS = 100         # 100 for GPU Server
INPUT_IMAGE_SIZE_PIXELS = 512
INPUT_SHAPE_IMAGE = [512, 512, 1]
LEARNING_RATE=0.00001
VAL_SUBSPLITS = 5

NUM_OF_EXPERIMENTS = 1      #How many times to run each task

EARLY_STOPPING_PATIENCE = 20 # 3 is the default value in tensorflow

SEED = 24

INITIAL_PERCENTAGE = 100

MODEL_NAME = f'Model training on ECHONET with Tversky loss'

# if WITH_DROPOUT:
#     MODEL_NAME = f'{MODEL_NAME} with Dropout'
    
SAVE_FOLDER = 'Trained Models'

# Functions for reading images, displaying predictions, and callback

In [4]:
def parse_image(img_path: str) -> dict:
    """Load an image and its annotation (mask) and return a dictionary.

    img_path : str : Image (not the mask) location.
    dict: Dictionary mapping an image and its annotation.
    """
    image = tf.io.read_file(img_path)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.convert_image_dtype(image, tf.uint8)

    # For one Image path:
    # .../dataset/images/training/im_00000001.jpg
    # Its corresponding annotation path is:
    # .../dataset/annotations/training/im_00000001.png
    mask_path = tf.strings.regex_replace(img_path, "images", "annotations_binary")
    mask_path = tf.strings.regex_replace(mask_path, "jpg", "png")
    mask = tf.io.read_file(mask_path)
    # The masks contain a class index for each pixels
    mask = tf.image.decode_png(mask, channels=1)
    # In scene parsing, "not labeled" = 255
    # But it will mess up with our N_CLASS = 150
    # Since 255 means the 255th class, which doesn't exist
    #mask = tf.where(mask == 255, np.dtype('uint8').type(0), mask)
    # Note that we have to convert the new value (0)
    # With the same dtype than the tensor itself    

    return {'image': image, 'segmentation_mask': mask}

In [5]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0  
  return input_image, input_mask

In [6]:
def load_image(datapoint):
  input_image = tf.image.resize(datapoint['image'], (INPUT_IMAGE_SIZE_PIXELS, INPUT_IMAGE_SIZE_PIXELS))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (INPUT_IMAGE_SIZE_PIXELS, INPUT_IMAGE_SIZE_PIXELS))

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [7]:
#Visualize an image example and its corresponding 
#mask from the dataset.

def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [8]:
def create_mask(pred_mask):
  #pred_mask = tf.argmax(pred_mask, axis=-1)
  #pred_mask = pred_mask[..., tf.newaxis]
  pred_mask = tf.greater(pred_mask, 0.5)
  pred_mask = tf.dtypes.cast(pred_mask, tf.float32)
  #pred_mask = pred_mask[..., tf.newaxis]
  pred_mask = pred_mask[0]
  return pred_mask

def create_mask_temp(pred_mask):
  #pred_mask = tf.argmax(pred_mask, axis=-1)
  #pred_mask = pred_mask[..., tf.newaxis]
  pred_mask = tf.greater(pred_mask, 0.5)
  pred_mask = tf.dtypes.cast(pred_mask, tf.float32)
  #pred_mask = pred_mask[..., tf.newaxis]  
  return pred_mask

In [9]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [10]:
def create_mask2(pred_mask):
  #pred_mask = tf.argmax(pred_mask, axis=-1)
  #pred_mask = pred_mask[..., tf.newaxis]
  pred_mask = tf.greater(pred_mask, 0.5)
  pred_mask = tf.dtypes.cast(pred_mask, tf.float32)
  #pred_mask = pred_mask[..., tf.newaxis]
  pred_mask = pred_mask[1]
  return pred_mask

def show_predictions_manual(image, mask, predicted): 
    display([image[0], mask[0], create_mask(predicted)])
    
def show_predictions_manual2(image, mask, predicted): 
    display([image[0], mask[0], create_mask2(predicted)])
  

In [11]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

In [12]:
#How to call above functions:

#for images, masks in train.take(2):
#  sample_image, sample_mask = images, masks
#  display([sample_image, sample_mask])


#show_predictions(test_data, 3)

# Build the model

In [13]:
import keras
from tensorflow.keras import backend as K 
from tensorflow.keras.optimizers import Adam

#For plotting a model:
#tf.keras.utils.plot_model(model, show_shapes=True)

### Contracting (downward) path

The (height, width, length) of the input gets smaller as you move down this path, and the number of channels increases.

### Expanding (upward) Path

The image's (height, width, length) all get larger in the expanding path.

The "depth" of your U-Net is equal to the number of down-convolutions.

# Original UNET:


In [14]:
initializer = tf.keras.initializers.HeNormal(seed=SEED)

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

#shape = (2, 2)
#initializer = tf.initializers.he_normal(seed = SEED)
#var = tf.Variable(initializer(shape=shape))

In [15]:
def conv_block(input_tensor, num_filters):
  encoder = tf.keras.layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
  encoder = tf.keras.layers.BatchNormalization()(encoder)
  encoder = tf.keras.layers.Activation('relu')(encoder)
  encoder = tf.keras.layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
  encoder = tf.keras.layers.BatchNormalization()(encoder)
  encoder = tf.keras.layers.Activation('relu')(encoder)
  return encoder

def encoder_block(input_tensor, num_filters):
  encoder = conv_block(input_tensor, num_filters)
  encoder_pool = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
  
  return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
  decoder = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
  decoder = tf.keras.layers.concatenate([concat_tensor, decoder], axis=-1)
  decoder = tf.keras.layers.BatchNormalization()(decoder)
  decoder = tf.keras.layers.Activation('relu')(decoder)
  decoder = tf.keras.layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
  decoder = tf.keras.layers.BatchNormalization()(decoder)
  decoder = tf.keras.layers.Activation('relu')(decoder)
  decoder = tf.keras.layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
  decoder = tf.keras.layers.BatchNormalization()(decoder)
  decoder = tf.keras.layers.Activation('relu')(decoder)
  return decoder

In [16]:
from loss_functions import *
loss_funcs = Semantic_loss_functions()

def unet_model():
    inputs = tf.keras.layers.Input(shape=INPUT_SHAPE_IMAGE)
    encoder0_pool, encoder0 = encoder_block(inputs, 32)
    encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64)
    encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128)
    encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256)
    encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512)
    center = conv_block(encoder4_pool, 1024)
    decoder4 = decoder_block(center, encoder4, 512)
    decoder3 = decoder_block(decoder4, encoder3, 256)
    decoder2 = decoder_block(decoder3, encoder2, 128)
    decoder1 = decoder_block(decoder2, encoder1, 64)
    decoder0 = decoder_block(decoder1, encoder0, 32)    
    outputs = tf.keras.layers.Conv2D(OUTPUT_CHANNELS, (1, 1), activation='sigmoid')(decoder0)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(optimizer=Adam(lr=LEARNING_RATE),
                  loss=loss_funcs.tversky_loss,
                  metrics=['accuracy'], run_eagerly=True
                 )  
    
    return model

semantic loss functions initialized


# Prepare Files

In [17]:
print(train_dir)
print(val_dir)
print(test_dir)

/tf/notebooks/Data/echonet frames split for model training/images/training
/tf/notebooks/Data/echonet frames split for model training/images/validation
/tf/notebooks/Data/echonet frames split for model training/images/testing


In [18]:
train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)
val_files = os.listdir(val_dir)

for i in range(0, len(train_files)):
    filename = train_files[i]
    train_files[i] = os.path.join(train_dir, filename)
    
for i in range(0, len(test_files)):
    filename = test_files[i]
    test_files[i] = os.path.join(test_dir, filename)
    
for i in range(0, len(val_files)):
    filename = val_files[i]
    val_files[i] = os.path.join(val_dir, filename)

#Use same order as folder
#random.shuffle(train_files)
#random.shuffle(test_files)
#random.shuffle(val_files)

print(f'Training File Count: {len(train_files)}')
print(f'Validation File Count: {len(val_files)}')
print(f'Test File Size: {len(test_files)}') 

Training File Count: 10754
Validation File Count: 3566
Test File Size: 3580


# My Training Functions, Multi-Model and Single-Model

In [19]:
#Note that I am reading files in the default folder order(alphabetical),
#so we assume the files are always in this order.
#The reason for this is because the shared model will be used in different
#experiments and when the results are plotted, it should
#have the same staring point at the specified initial percentage.

training_set_size = int(float(INITIAL_PERCENTAGE/100.0) * len(train_files))
training_files = train_files[ : training_set_size]

model = unet_model()

TRAIN_LENGTH = len(training_files)        
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE       
VALIDATION_STEPS = len(val_files) // BATCH_SIZE // VAL_SUBSPLITS

print(f'Number of training files at {INITIAL_PERCENTAGE}% = {TRAIN_LENGTH}')
print(f'Steps per Epoch = {STEPS_PER_EPOCH}')
print(f'Validation Steps = {VALIDATION_STEPS}')

#Load Data into tensorflow dataset and map images:
val_imgs = tf.data.Dataset.from_tensor_slices(val_files)
val_set = val_imgs.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)
val = val_set.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
val_data = val.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

train_imgs = tf.data.Dataset.from_tensor_slices(training_files)
train_set = train_imgs.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)
train = train_set.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
train_data = (train
            .cache()
            .shuffle(BUFFER_SIZE)
            .batch(BATCH_SIZE)
            .repeat()
            #.map(Augment())  no image augmentation for now
            .prefetch(buffer_size=tf.data.AUTOTUNE))

callback_early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING_PATIENCE, verbose=1, restore_best_weights=True)


model_history = model.fit(train_data, 
                          epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH, 
                          validation_steps=VALIDATION_STEPS, 
                          validation_data=val_data,
                          callbacks=[callback_early_stopping])

model.save(f'{SAVE_FOLDER}/{MODEL_NAME}')


/usr/local/lib/python3.6/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Number of training files at 100% = 10754
Steps per Epoch = 672
Validation Steps = 44
Epoch 1/100
672/672 [==============================] - 380s 557ms/step - loss: 0.4875 - accuracy: 0.8061 - val_loss: 0.3587 - val_accuracy: 0.9473
Epoch 2/100
672/672 [==============================] - 375s 556ms/step - loss: 0.3092 - accuracy: 0.9640 - val_loss: 0.2919 - val_accuracy: 0.9654
Epoch 3/100
672/672 [==============================] - 374s 556ms/step - loss: 0.2800 - accuracy: 0.9679 - val_loss: 0.2777 - val_accuracy: 0.9716
Epoch 4/100
672/672 [==============================] - 374s 556ms/step - loss: 0.2602 - accuracy: 0.9699 - val_loss: 0.2535 - val_accuracy: 0.9736
Epoch 5/100
672/672 [==============================] - 373s 556ms/step - loss: 0.2440 - accuracy: 0.9711 - val_loss: 0.2451 - val_accuracy: 0.9643
Epoch 6/100
672/672 [==============================] - 373s 556ms/step - loss: 0.2282 - accuracy: 0.9726 - val_loss: 0.2232 - val_accuracy: 0.9723
Epoch 7/100
672/672 [============

Epoch 56/100
672/672 [==============================] - 373s 555ms/step - loss: 0.0210 - accuracy: 0.9853 - val_loss: 0.0778 - val_accuracy: 0.9770
Epoch 57/100
672/672 [==============================] - 373s 555ms/step - loss: 0.0205 - accuracy: 0.9854 - val_loss: 0.0709 - val_accuracy: 0.9768
Epoch 58/100
672/672 [==============================] - 373s 555ms/step - loss: 0.0205 - accuracy: 0.9854 - val_loss: 0.0785 - val_accuracy: 0.9765
Epoch 59/100
672/672 [==============================] - 373s 555ms/step - loss: 0.0203 - accuracy: 0.9854 - val_loss: 0.0786 - val_accuracy: 0.9768
Epoch 60/100
672/672 [==============================] - 373s 556ms/step - loss: 0.0204 - accuracy: 0.9854 - val_loss: 0.0782 - val_accuracy: 0.9764
Epoch 61/100
672/672 [==============================] - 373s 555ms/step - loss: 0.0200 - accuracy: 0.9854 - val_loss: 0.0794 - val_accuracy: 0.9763
Epoch 62/100
672/672 [==============================] - 373s 555ms/step - loss: 0.0201 - accuracy: 0.9854 - val_